In [5]:
import os
import scipy.io

# Load .mat annotations
meta_path = os.path.join("..", "data", "SUNRGBDMeta3DBB_v2.mat")
meta = scipy.io.loadmat(meta_path, struct_as_record=False, squeeze_me=True)

# List of samples
samples = meta["SUNRGBDMeta"]
print("✅ Loaded", len(samples), "samples")


✅ Loaded 10335 samples


In [6]:
# Print available attributes for debug
sample = samples[0]
box3d = sample.groundtruth3DBB[0] if isinstance(sample.groundtruth3DBB, np.ndarray) else sample.groundtruth3DBB

print("Available attributes:")
for attr in dir(box3d):
    if not attr.startswith("__"):
        print(attr)


Available attributes:
_fieldnames
basis
centroid
classname
coeffs
label
orientation
sequenceName


In [10]:
import open3d as o3d
import numpy as np

# Get first sample and first object
sample = samples[0]
box = sample.groundtruth3DBB[0]

# Extraer atributos
basis = np.array(box.basis)         # 3x3 rotation matrix
centroid = np.array(box.centroid)   # center (x, y, z)
coeffs = np.array(box.coeffs)       # half-sizes (l/2, w/2, h/2)

# Calcular dimensiones completas
extent = coeffs * 2                 # (l, w, h)

# Crear caja 3D orientada
obb = o3d.geometry.OrientedBoundingBox(center=centroid, R=basis, extent=extent)
obb.color = (1, 0, 0)  # rojo

# Mostrar
o3d.visualization.draw_geometries([obb], window_name="Sample 3D Box")


In [11]:
import open3d as o3d
import numpy as np

# Get first sample
sample = samples[0]
boxes = sample.groundtruth3DBB

print(f"Total boxes in sample: {len(boxes)}")

geometries = []

for box in boxes:
    basis = np.array(box.basis)
    centroid = np.array(box.centroid)
    coeffs = np.array(box.coeffs)
    extent = coeffs * 2

    obb = o3d.geometry.OrientedBoundingBox(center=centroid, R=basis, extent=extent)
    obb.color = (0, 1, 0)  # green
    geometries.append(obb)

# Visualize all bounding boxes
o3d.visualization.draw_geometries(geometries, window_name="All 3D Bounding Boxes in Sample")


Total boxes in sample: 6


In [12]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import hashlib

# Simple hashing function to get consistent RGB values per class
def get_color_from_class(classname):
    hash_val = int(hashlib.md5(classname.encode()).hexdigest(), 16)
    r = ((hash_val >> 16) & 255) / 255.0
    g = ((hash_val >> 8) & 255) / 255.0
    b = (hash_val & 255) / 255.0
    return (r, g, b)

# First sample
sample = samples[0]
boxes = sample.groundtruth3DBB

geometries = []

for box in boxes:
    basis = np.array(box.basis)
    centroid = np.array(box.centroid)
    coeffs = np.array(box.coeffs)
    extent = coeffs * 2
    classname = box.classname

    obb = o3d.geometry.OrientedBoundingBox(center=centroid, R=basis, extent=extent)
    obb.color = get_color_from_class(classname)
    geometries.append(obb)

# Display all 3D bounding boxes with class-based color
o3d.visualization.draw_geometries(geometries, window_name="3D Boxes with Class Colors")
